### Getting info from Google News (NYTimes)


In [ ]:
import http.client

conn = http.client.HTTPSConnection("google-news.p.rapidapi.com")

headers = {
    'x-rapidapi-key': "83669c4ebamsh964bc077fea4c66p10dbedjsncb5951952abc",
    'x-rapidapi-host': "google-news.p.rapidapi.com"
    }

conn.request("GET", "/v1/source_search?source=nytimes.com&lang=en&country=US&from=2021-03-11&to=2021-03-12", headers=headers)

res = conn.getresponse()
data = res.read()

print(data.decode("utf-8"))


Since the full article is not included, the link is taken from the articles and will be opened in another cell

In [ ]:
import json
import requests

jsonNews = json.loads(data.decode("utf-8"))
jsonNews = jsonNews.get('articles')
links = []
requestedLinks = []

for i in range(len(jsonNews)):
    links.append(jsonNews[i].get('link'))
    temp = requests.get(links[i])
    requestedLinks.append(temp.content)

Here, BeautifulSoup is imported and is used to parse the content of the links

In [ ]:
from bs4 import BeautifulSoup

soup = []
for i in range(len(links)):
    soup.append(BeautifulSoup(requestedLinks[i], 'html.parser'))



The needed info is taken from the initial requested data, and the authors and full article are scraped from the page in the link. The info is appended into an array of json objects.

In [ ]:
jsonNewsExtracted = []
for i in range(len(jsonNews)):
    date = jsonNews[i].get('published')
    title = jsonNews[i].get('title')
    allAuthors = ""
    authors = soup[i].find_all("span", itemprop="name")
    for i in range (len(authors)):
        if i != len(authors)-1:
            allAuthors = allAuthors + authors[i].text + ', '
        else:
            allAuthors = allAuthors + authors[i].text
    try:
        full_article = soup[i].find(id = 'story').find('section').text
    except:
        pass
    jsonNewsExtracted.append({
            "date": date,
            "title": title,
            "full_article": full_article,
            "authors": allAuthors
        })

The json object array is exported as news.json

In [ ]:
json_dump = json.dumps(jsonNewsExtracted, indent = 4) 

with open("news.json", "w") as outfile:
    outfile.write(json_dump)